In [ ]:
from flask import jsonify
from flask import Flask, request, render_template, send_from_directory
from ocr import predict
from keras.models import model_from_json
import tensorflow as tf
from keras import backend as K
import os
import tornado.wsgi
import tornado.httpserver
import sqlite3
from flask import Flask, render_template, request, redirect, session, url_for
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
REPO_DIRNAME = os.path.dirname(os.path.abspath(__file__))


In [ ]:

############################### Database Creation ########################################

# Set the database URI to a SQLite database named "users.db"
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'

# Set the secret key for the Flask application
app.secret_key = "ThisIsNotASecret:p"

# Create an instance of SQLAlchemy and bind it to the Flask application
db = SQLAlchemy(app)

# Define a User class representing a table in the database
class User(db.Model):
    # Define the id column as an integer primary key
    id = db.Column(db.Integer, primary_key=True)
    
    # Define the first_name column as a string
    first_name = db.Column(db.String(100))
    
    # Define the last_name column as a string
    last_name = db.Column(db.String(100))
    
    # Define the email column as a unique string
    email = db.Column(db.String(100), unique=True)
    
    # Define the username column as a unique string
    username = db.Column(db.String(100), unique=True)
    
    # Define the password column as a string
    password = db.Column(db.String(100))

    # Define the constructor for the User class
    def __init__(self, first_name, last_name, email, username, password):
        # Assign the first_name parameter to the first_name instance variable
        self.first_name = first_name
        
        # Assign the last_name parameter to the last_name instance variable
        self.last_name = last_name
        
        # Assign the email parameter to the email instance variable
        self.email = email
        
        # Assign the username parameter to the username instance variable
        self.username = username
        
        # Assign the password parameter to the password instance variable
        self.password = password

In [ ]:
############################################################################################

# Define a class named ImagenetClassifier
class ImagenetClassifier(object):
    def __init__(self):
        # Print a message to indicate that the model is being loaded
        print('LOAD MODEL ...')
        
        # Load the model architecture from a JSON file
        with open('Resource/model_prediction.json', 'r') as f:
            self.w_model_predict = model_from_json(f.read())
        
        # Load the model weights from an H5 file
        self.w_model_predict.load_weights('Resource/iam_words--17--1.887.h5')
    
    def predict_image(self, image_filename):
        try:
            # Perform prediction using the loaded model
            with graph.as_default():
                pred_text_model_word = predict(self.w_model_predict, image_filename)
            
            # Return the prediction result
            return pred_text_model_word
        
        except Exception as err:
            # Print an error message if there's an exception during prediction
            print('Prediction error: ', err)
            
            # Return a tuple indicating the error
            return (False, 'The file selected is not supported! '
                           'Please select another one.')


# Create an instance of ImagenetClassifier and assign it to ocr_model
global ocr_model
ocr_model = ImagenetClassifier()

# Create a TensorFlow graph and assign it to graph
global graph
graph = tf.compat.v1.get_default_graph()


In [ ]:
###################################  USER MANAGEMENT #########################################

@app.route('/')
def index():
    # Render the index.html template
    return render_template('index.html')

@app.route('/main')
def main():
    # Render the main.html template
    return render_template('main.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        # Retrieve form data submitted by the user
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        email = request.form['email']
        username = request.form['username']
        password = request.form['password']

        # Check if email or username already exists
        existing_user = User.query.filter((User.email == email) | (User.username == username)).first()
        if existing_user:
            # Render the register.html template with an error message
            return render_template('register.html', message='Email or username already exists')

        # Create a new User instance
        user = User(first_name=first_name, last_name=last_name, email=email, username=username, password=password)
        
        # Add the user to the database session
        db.session.add(user)
        
        # Commit the changes to the database
        db.session.commit()
        
        # Redirect the user to the login page
        return redirect(url_for('login'))
    
    # For a GET request, render the register.html template
    return render_template('register.html')


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Retrieve form data submitted by the user
        username = request.form['username']
        password = request.form['password']
        
        # Query the database to find a matching User object
        user = User.query.filter_by(username=username, password=password).first()
        
        if user:
            # Set the 'logged_in' key in the session
            session['logged_in'] = True
            
            # Redirect the user to the home page
            return redirect(url_for('home'))
        else:
            # Render the login.html template with an error message
            return render_template('login.html', message='Invalid login credentials')
    
    # For a GET request, render the login.html template
    return render_template('login.html')


@app.route('/logout')
def logout():
    # Remove the 'logged_in' key from the session
    session.pop('logged_in', None)
    
    # Redirect the user to the login page
    return redirect(url_for('login'))


@app.route('/home')
def home():
    if 'logged_in' in session:
        # Render the home.html template
        return render_template('home.html')
    
    # If not logged in, redirect the user to the login page
    return redirect(url_for('login'))

In [ ]:

##################################################################################################

target = os.path.join(REPO_DIRNAME, 'images')
assert os.path.isdir(target), f"{target} is not a valid directory"

# Define the "/upload" route to handle file uploads
@app.route("/upload", methods=["POST"])
def upload():
    target = os.path.join(REPO_DIRNAME, 'images')
    
    # Create the "images" directory if it doesn't exist
    if not os.path.isdir(target):
        os.mkdir(target)
    
    # Retrieve the uploaded files from the request
    print(request.files.getlist("file"))
    
    # Iterate over each uploaded file
    for upload in request.files.getlist("file"):
        print(upload)
        print("{} is the file name".format(upload.filename))
        filename = upload.filename
        
        # Set the destination path for saving the file
        destination = "/".join([target, filename])
        
        print("Accept incoming file:", filename)
        print("Save it to:", destination)
        
        # Save the file to the destination path
        upload.save(destination)
    
    # Perform some prediction or processing on the uploaded image
    pred_text_model_word = ocr_model.predict_image(destination)
    
    # Render the index.html template with the prediction result and image name
    return render_template("index.html", predict1=pred_text_model_word, image_name=filename)

# Define the "/upload/<filename>" route to serve uploaded images
@app.route('/upload/<filename>')
def send_image(filename):
    # Send the uploaded image from the "images" directory
    return send_from_directory("images", filename)

# Start the Tornado server
def start_tornado(app, port=5000):
    http_server = tornado.httpserver.HTTPServer(
        tornado.wsgi.WSGIContainer(app))
    http_server.listen(port)
    print("Tornado server starting on port {}".format(port))
    tornado.ioloop.IOLoop.instance().start()

if __name__ == "__main__":
    # Start the Tornado server with the Flask app
    start_tornado(app)